In [57]:
import string
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql import functions
from pyspark.sql import types

In [2]:
# create sparksession
ss = SparkSession \
    .builder \
    .appName("Manipulation DataFrame") \
    .master("local[2]") \
    .getOrCreate()

In [3]:
# load data
df = ss.read.json("./data/data.json")

## 数据评估

### 显示数据类型
显示数据类型的方法，可以使用 `printSchema()` 以及 `dtypes` 属性。前者是返回树形结构，后者是得到一个键值形式列表

In [5]:
# 显示 Schema
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- location: struct (nullable = true)
 |    |-- altitude: string (nullable = true)
 |    |-- country: string (nullable = true)
 |    |-- exact_location: long (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- indoor: long (nullable = true)
 |    |-- latitude: string (nullable = true)
 |    |-- longitude: string (nullable = true)
 |-- sampling_rate: string (nullable = true)
 |-- sensor: struct (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- pin: string (nullable = true)
 |    |-- sensor_type: struct (nullable = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- manufacturer: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |-- sensordatavalues: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- value: string (nullable = true)
 |    |    |-- value_type: string (nullable = true)
 |-- timestamp: string (nullab

In [11]:
df.dtypes

[('id', 'bigint'),
 ('location',
  'struct<altitude:string,country:string,exact_location:bigint,id:bigint,indoor:bigint,latitude:string,longitude:string>'),
 ('sampling_rate', 'string'),
 ('sensor',
  'struct<id:bigint,pin:string,sensor_type:struct<id:bigint,manufacturer:string,name:string>>'),
 ('sensordatavalues',
  'array<struct<id:bigint,value:string,value_type:string>>'),
 ('timestamp', 'string')]

### 显示DataFrame 维度数据

In [95]:
df.count(), len(df.columns)

(35527, 6)

### 显示前几行数据
有多种方法显示前几行数据，`head()`以及特殊的方法 `first()`、`show()` 等。需要注意返回的结果类型不是完全相同的。

In [6]:
df.head(2)

[Row(id=5756852209, location=Row(altitude='104.9', country='UA', exact_location=0, id=22256, indoor=1, latitude='50.51', longitude='30.798'), sampling_rate=None, sensor=Row(id=36214, pin='7', sensor_type=Row(id=9, manufacturer='various', name='DHT22')), sensordatavalues=[Row(id=12224991603, value='10.00', value_type='temperature'), Row(id=12224991604, value='50.70', value_type='humidity')], timestamp='2019-12-13 11:10:02'),
 Row(id=5756852208, location=Row(altitude='111.8', country='GB', exact_location=1, id=21003, indoor=0, latitude='53.87869338867', longitude='-1.45841360092'), sampling_rate=None, sensor=Row(id=34792, pin='11', sensor_type=Row(id=17, manufacturer='Bosch', name='BME280')), sensordatavalues=[Row(id=12224991602, value='96357.16', value_type='pressure'), Row(id=12224991605, value='1.93', value_type='temperature'), Row(id=12224991606, value='100.00', value_type='humidity'), Row(id=None, value='97702.38', value_type='pressure_at_sealevel')], timestamp='2019-12-13 11:10:02'

In [7]:
df.first()

Row(id=5756852209, location=Row(altitude='104.9', country='UA', exact_location=0, id=22256, indoor=1, latitude='50.51', longitude='30.798'), sampling_rate=None, sensor=Row(id=36214, pin='7', sensor_type=Row(id=9, manufacturer='various', name='DHT22')), sensordatavalues=[Row(id=12224991603, value='10.00', value_type='temperature'), Row(id=12224991604, value='50.70', value_type='humidity')], timestamp='2019-12-13 11:10:02')

In [9]:
df.show(n=2)

+----------+--------------------+-------------+--------------------+--------------------+-------------------+
|        id|            location|sampling_rate|              sensor|    sensordatavalues|          timestamp|
+----------+--------------------+-------------+--------------------+--------------------+-------------------+
|5756852209|[104.9, UA, 0, 22...|         null|[36214, 7, [9, va...|[[12224991603, 10...|2019-12-13 11:10:02|
|5756852208|[111.8, GB, 1, 21...|         null|[34792, 11, [17, ...|[[12224991602, 96...|2019-12-13 11:10:02|
+----------+--------------------+-------------+--------------------+--------------------+-------------------+
only showing top 2 rows



### 显示数据列名称
通过显示 `schema` 和 `dtypes` 可以显示列名称，需要直接得到列名称可以直接通过 `columns` 属性获取

In [13]:
df.columns

['id', 'location', 'sampling_rate', 'sensor', 'sensordatavalues', 'timestamp']

### 缺失值统计
DataFrame 没有自带的检测是否为缺失值方法，需要通过调用 `functions` 模块中的方法可以进行统计缺失值，大致步骤如下：
1. 通过 `isnull()` 判断是否为缺失值，此外需要注意该方法和 pandas 有差异——pandas 没有区分 NAN 和 NULL
2. 使用 `cast()` 方法将数据类型转换为 `integer`。该步骤可以通过 [`when()`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.when)  和 [`otherwise()`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.otherwise)方法完成
3. 通过 `sum()` 方法累计求和

In [15]:
# 使用 pandas 的 isnull 方法判断
pd.isnull(np.nan), pd.isnull(None)

(True, True)

In [14]:
df.select(
    functions.sum(
        functions.isnull("sampling_rate").cast("integer")
    ).alias("sampling_rate")
).show()

+-------------+
|sampling_rate|
+-------------+
|        35527|
+-------------+



In [13]:
# 第二种方法，通过 when 和 otherwise 完成——实际内核是 SQL 中 CASE WHEN 方法
# 注意 when 和 otherwise 可以链式表达而不需要额外的
df.select(
    functions.when(
        functions.isnull("sampling_rate"), 1
    ).otherwise(0).alias("sampling_rate")
).select(functions.sum("sampling_rate").alias("Miss_value")).show()

+----------+
|Miss_value|
+----------+
|     35527|
+----------+



In [20]:
# 计算 columns 的缺失值数量
df.select([functions.sum(functions.when(functions.isnull(c), 1).otherwise(0)).alias(c) for c in df.columns]).show()

+---+--------+-------------+------+----------------+---------+
| id|location|sampling_rate|sensor|sensordatavalues|timestamp|
+---+--------+-------------+------+----------------+---------+
|  0|       0|        35527|     0|               0|        0|
+---+--------+-------------+------+----------------+---------+



In [27]:
# 替换方法计算 columns 缺失值
df.select(
        [functions.sum(
            functions.isnull(x).cast("integer")
        ).alias(x)
         for x in df.columns
        ]).show()

+---+--------+-------------+------+----------------+---------+
| id|location|sampling_rate|sensor|sensordatavalues|timestamp|
+---+--------+-------------+------+----------------+---------+
|  0|       0|        35527|     0|               0|        0|
+---+--------+-------------+------+----------------+---------+



In [36]:
# 通过 UDF 计算，需要先分别检验各列中缺失值，在进行计算
def check_missing(x):
    if x is None:
        return 1
    else:
        return 0
        
missing_count = functions.udf(lambda x: check_missing(x), types.IntegerType())

In [40]:
# 计算单一列
df.select(missing_count("sampling_rate").alias("test")).select(functions.sum("test")).show()

+---------+
|sum(test)|
+---------+
|    35527|
+---------+



In [44]:
df.select([
    missing_count(x).alias(x) for x in df.columns
    ]).select([
        functions.sum(x).alias(x) for x in df.columns
    ]).show()

+---+--------+-------------+------+----------------+---------+
| id|location|sampling_rate|sensor|sensordatavalues|timestamp|
+---+--------+-------------+------+----------------+---------+
|  0|       0|        35527|     0|               0|        0|
+---+--------+-------------+------+----------------+---------+



In [50]:
# 使用 * 可以快速筛选列，并且可以定制化通配符筛选
df.select([
    missing_count(x).alias(x) for x in df.columns
    ]).select([
        functions.sum(x).alias(x) for x in df.columns
    ]).select("*").show()

+---+--------+-------------+------+----------------+---------+
| id|location|sampling_rate|sensor|sensordatavalues|timestamp|
+---+--------+-------------+------+----------------+---------+
|  0|       0|        35527|     0|               0|        0|
+---+--------+-------------+------+----------------+---------+



### 显示基本的统计信息
pandas 中可以通过 `DataFrame.describe()` 查看各列的统计信息。Spark 也支持该功能，可以使用 `summary()` 和 `describe()` 方法查询

In [46]:
df.describe().show()

+-------+-------------------+-------------+-------------------+
|summary|                 id|sampling_rate|          timestamp|
+-------+-------------------+-------------+-------------------+
|  count|              35527|            0|              35527|
|   mean|5.756834376153911E9|         null|               null|
| stddev| 10295.038530963458|         null|               null|
|    min|         5756816550|         null|2019-12-13 11:04:53|
|    max|         5756852209|         null|2019-12-13 11:10:02|
+-------+-------------------+-------------+-------------------+



In [48]:
df.summary().show()

+-------+-------------------+-------------+-------------------+
|summary|                 id|sampling_rate|          timestamp|
+-------+-------------------+-------------+-------------------+
|  count|              35527|            0|              35527|
|   mean|5.756834376153911E9|         null|               null|
| stddev| 10295.038530963458|         null|               null|
|    min|         5756816550|         null|2019-12-13 11:04:53|
|    25%|         5756825456|         null|               null|
|    50%|         5756834377|         null|               null|
|    75%|         5756843280|         null|               null|
|    max|         5756852209|         null|2019-12-13 11:10:02|
+-------+-------------------+-------------+-------------------+



### 数据冗余统计
Spark 统计数据冗余，和 Pandas 存在差异——pandas 直接提供了 `DataFrame.duplicate()` 方法判断数据是否冗余。Spark 需要通过多步骤完成，[countDistinc](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.countDistinct) 统计唯一值数量——其中非数值数据也是统计为一个唯一值， [count](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.count) 统计数据长度。

当需要以多字段判断是否有冗余值时，需要通过 groupby 计数数量大于 1 的组合。

In [105]:
list(range(20))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [121]:
# 创建数据
schema = types.StructType([
    types.StructField("col1", types.StringType()),
    types.StructField("col2", types.FloatType()),
    types.StructField("col3", types.IntegerType()),
    types.StructField("col4", types.FloatType())
])

test = ss.createDataFrame(
    list(zip(
        np.random.choice(list(string.ascii_uppercase), size=26).tolist(), 
        np.random.normal(0, 10,  size=26).tolist(), 
        np.random.randint(0, 20,  size=26).tolist(),
        np.random.choice(list(range(4)) + [np.nan], size=26).tolist())), schema=schema
)

test.persist()

test.head(2)

[Row(col1='R', col2=0.5188218355178833, col3=13, col4=0.0),
 Row(col1='X', col2=4.821608543395996, col3=4, col4=3.0)]

In [115]:
# 统计各列的长度
test.select([functions.count(x).alias(x) for x in test.columns]).show()

+----+----+----+----+
|col1|col2|col3|col4|
+----+----+----+----+
|  26|  26|  26|  26|
+----+----+----+----+



In [116]:
# 该方法将缺失值也是统计为了唯一值
test.select("col4").distinct().show()

+----+
|col4|
+----+
| 2.0|
| 3.0|
| 1.0|
| NaN|
| 0.0|
+----+



In [117]:
# 统计各列唯一值数量
test.select([functions.countDistinct(x).alias(x) for x in test.columns]).show()

+----+----+----+----+
|col1|col2|col3|col4|
+----+----+----+----+
|  19|  26|  14|   5|
+----+----+----+----+



In [118]:
# 通过两个长度值减去唯一值长度计算数据冗余数量
test.select([(functions.count(x) - functions.countDistinct(x)).alias(x) for x in test.columns]).show()

+----+----+----+----+
|col1|col2|col3|col4|
+----+----+----+----+
|   7|   0|  12|  21|
+----+----+----+----+



In [56]:
df.distinct().select([functions.count(x) for x in df.columns]).show()

+---------+---------------+--------------------+-------------+-----------------------+----------------+
|count(id)|count(location)|count(sampling_rate)|count(sensor)|count(sensordatavalues)|count(timestamp)|
+---------+---------------+--------------------+-------------+-----------------------+----------------+
|    35527|          35527|                   0|        35527|                  35527|           35527|
+---------+---------------+--------------------+-------------+-----------------------+----------------+



In [142]:
# 统计个组合下的冗余数量
test.groupBy("col1", "col4").count().filter("count > 1" ).show()

+----+----+-----+
|col1|col4|count|
+----+----+-----+
|   D| NaN|    2|
+----+----+-----+



## 数据清理

### 类型转换

* 日期或时间戳转换为日期字符串 使用 [date_format()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.date_format) 方法进行转换
* 日期字符串转换转换为日期或时间戳 使用 [to_date()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.to_date) 和 [to_timestamp](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.to_timestamp) 方法进行转换，如果没有设置 `format` 参数前者相当于使用 `col.cast("date")`后者相当于使用了 `col.cast("timestamp")`。需要注意两者在使用的格式需要依据 [Java 模式格式](https://docs.oracle.com/javase/tutorial/i18n/format/simpleDateFormat.html)
* 比较通用的转换方法是使用 [cast()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.cast) 进行类型转换——思路和 SQL 中 `cast()` 相似。此外还有类似的 [astype()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.astype)

需要注意第三种方法是 `Column` 的方法，在使用的时候最好强制申明 `df.select(functions.col("datetime").cast("string"))`

In [148]:
# 转换为时间戳
df.select(functions.to_timestamp("timestamp", "yyyy-MM-dd hh:mm:ss").alias("time")).dtypes

[('time', 'timestamp')]

In [149]:
# 时间戳转换为字符串
df.select(
    functions.to_timestamp("timestamp", "yyyy-MM-dd hh:mm:ss").alias("time")
).select(
    functions.date_format("time", "yyyy-MM-dd hh:mm:ss").alias("time")
).dtypes

[('time', 'string')]

In [158]:
# 使用 cast 方法
test.select(functions.col("col2").cast("string")).dtypes

[('col2', 'string')]

In [163]:
# 使用 astype 方法
df.select(functions.col("timestamp").cast("date").alias("date_")).head(2)

[Row(date_=datetime.date(2019, 12, 13)),
 Row(date_=datetime.date(2019, 12, 13))]

### 数据筛选
* 非缺失值条件筛选——使用 [isNotNull](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.isNotNull)